In [1]:
# !pip install tabulate
# !pip install sentence_transformers

In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
from tabulate import tabulate
import fasttext
import faiss
import re
from nltk.stem import WordNetLemmatizer
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import wordnet as wn
import time
import nltk

/home/jovyan/.local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-12-13 13:54:56.002471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-13 13:54:56.240723: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-13 13:54:57.058380: W tensorflow/compiler

In [3]:
animals = [
    "Aardvark",
    "Alligator",
    "Alpaca",
    "Ant",
    "Anteater",
    "Antelope",
    "Armadillo",
    "Baboon",
    "Badger",
    "Barracuda",
    "Bat",
    "Bear",
    "Beaver",
    "Bee",
    "Beetle",
    "Bird",
    "Bison",
    "Bobcat",
    "Buffalo",
    "Bull",
    "Butterfly",
    "Calf",
    "Camel",
    "Capybara",
    "Caribou",
    "Cat",
    "Caterpillar",
    "Centipede",
    "Chameleon",
    "Cheetah",
    "Chick",
    "Chicken",
    "Chimpanzee",
    "Chinchilla",
    "Chipmunk",
    "Cicada",
    "Clam",
    "Cockroach",
    # "Cougar", This can be a bad slur and is therefore removed
    "Cow",
    "Crab",
    "Crane",
    "Crayfish",
    "Cricket",
    "Crocodile",
    "Crow",
    "Cub",
    "Deer",
    "Dog",
    "Dolphin",
    "Donkey",
    "Dove",
    "Dragonfly",
    "Duck",
    "Duckling",
    "Eagle",
    "Elephant",
    "Elk",
    "Fawn",
    "Ferret",
    "Firefly",
    "Fish",
    "Flamingo",
    "Flea",
    "Fly",
    "Foal",
    "Fox",
    "Frog",
    "Gazelle",
    "Gecko",
    "Giraffe",
    "Goat",
    "Goose",
    "Gopher",
    "Gorilla",
    "Grasshopper",
    "Guinea Pig",
    "Hamster",
    "Hawk",
    "Hedgehog",
    "Hippo",
    "Horse",
    "Hummingbird",
    "Hyena",
    "Iguana",
    "Jaguar",
    "Jellyfish",
    "Kangaroo",
    "Kitten",
    "Koala",
    "Ladybug",
    "Lamb",
    "Lemming",
    "Lemur",
    "Leopard",
    "Lion",
    "Lizard",
    "Llama",
    "Lobster",
    "Lynx",
    "Manatee",
    "Manta Ray",
    "Meerkat",
    "Monkey",
    "Moose",
    "Mosquito",
    "Moth",
    "Mountain Lion",
    "Mouse",
    "Mule",
    "Muskrat",
    "Mussel",
    "Narwhal",
    "Newt",
    "Ocelot",
    "Octopus",
    "Opossum",
    "Orangutan",
    "Otter",
    "Owl",
    "Oyster",
    "Panda",
    "Parrot",
    "Pelican",
    "Penguin",
    "Pig",
    "Pigeon",
    "Piglet",
    "Piranha",
    "Platypus",
    "Pony",
    "Porcupine",
    "Prairie Dog",
    "Prawn",
    "Praying Mantis",
    "Puma",
    "Puppy",
    "Quail",
    "Rabbit",
    "Raccoon",
    "Rat",
    "Rhino",
    "Robin",
    "Rooster",
    "Salamander",
    "Scorpion",
    "Sea Lion",
    "Sea Urchin",
    "Seagull",
    "Seal",
    "Shark",
    "Sheep",
    "Shrew",
    "Shrimp",
    "Silverfish",
    "Skunk",
    "Sloth",
    "Slug",
    "Snail",
    "Snake",
    "Sparrow",
    "Spider",
    "Squid",
    "Squirrel",
    "Starfish",
    "Stork",
    "Swan",
    "Tadpole",
    "Termite",
    "Tick",
    "Tiger",
    "Toad",
    "Tortoise",
    "Turkey",
    "Turtle",
    "Wallaby",
    "Walrus",
    "Weasel",
    "Whale",
    "Wildebeest",
    "Wolf",
    "Wombat",
    "Woodpecker",
    "Worm",
    "Yak",
    "Zebra"
]

def find_antonym_soft(word, model, num_neighbours=300):
    neighbours = [w for _, w in model.get_nearest_neighbors(word, num_neighbours)]

    for nb in neighbours:
        # print('neighbour:', nb)
        for syn in wn.synsets(nb):
            # print(syn)
            for lemma in syn.lemmas():
                # print('lemma:', lemma.name())
                for ant in lemma.antonyms():
                    # print('ant:', ant.name())  # <-- get string, not Lemma object
                    return ant.name()
    return 'fantastic'  # no antonym found

def extreme_neighbors(word1, word2, model, scale=10.0, k=10):
    # word vectors
    v1 = model.get_word_vector(word1)
    v2 = model.get_word_vector(word2)

    # direction
    direction = v2 - v1

    # two extreme points
    extreme_neg = v1 - scale * direction
    extreme_pos = v2 + scale * direction

    # --- Build FAISS cosine index ---
    words = model.get_words()
    vectors = np.vstack([model.get_word_vector(w) for w in words])

    # normalize for cosine similarity
    vecs_norm = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    index = faiss.IndexFlatIP(vecs_norm.shape[1])
    index.add(vecs_norm)

    # normalize queries
    q_neg = extreme_neg / np.linalg.norm(extreme_neg)
    q_pos = extreme_pos / np.linalg.norm(extreme_pos)

    # search
    Dn, In = index.search(q_neg.reshape(1, -1), k)
    Dp, Ip = index.search(q_pos.reshape(1, -1), k)

    neg_neighbors = [(words[i], float(Dn[0][j])) for j, i in enumerate(In[0])]
    pos_neighbors = [(words[i], float(Dp[0][j])) for j, i in enumerate(Ip[0])]

    return neg_neighbors, pos_neighbors



def normalize_word(w, lemmatizer):
    # lowercase
    w = w.lower()

    # remove punctuation & non-letters
    w = re.sub(r"[^a-z]+", "", w)

    # lemmatize adjective form
    w = lemmatizer.lemmatize(w, pos='a')

    return w

def is_clean_word(w):
    """
    Accept only words made from letters, hyphens, apostrophes.
    Reject anything else (., /, _, digits, emojis…)
    """
    ALLOWED_PATTERN = re.compile(r"[A-Za-z'-]+")
    return ALLOWED_PATTERN.fullmatch(w) is not None


def normalize_word_strong(w, lemmatizer):
    original = w
    w = w.lower()

    # reject if any illegal character occurs
    if not is_clean_word(w):
        return "", original

    # collapse long repeats: cuuuute -> cute
    w = re.sub(r"(.)\1{2,}", r"\1", w)

    # remove double hyphens or apostrophes (e.g. -- or '')
    w = re.sub(r"[-']{2,}", lambda m: m.group(0)[0], w)

    # lemma for adjectives
    lemma = lemmatizer.lemmatize(w, pos='a')

    return lemma, original


def dedupe_word_list_strong(word_list, model, sim_threshold=0.80):
    cleaned = {}
    lemmatizer = WordNetLemmatizer()
    for word, score in word_list:
        base, original = normalize_word_strong(word, lemmatizer)

        if base == "":
            continue

        best_key = base

        for key in cleaned:
            # merge spelling variants
            if levenshtein_distance(base, key) <= 2:
                best_key = key
                break

            # merge semantically identical neighbors
            v1 = model.get_word_vector(base)
            v2 = model.get_word_vector(key)
            cos = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            if cos > sim_threshold:
                best_key = key
                break

        # keep highest score version
        if best_key not in cleaned or score > cleaned[best_key][1]:
            cleaned[best_key] = (word, score)

    # sort by score (high → low)
    return sorted(cleaned.values(), key=lambda x: x[1], reverse=True)

def find_extremes_fast(base_word, model):
    # model = fasttext.load_model('cc.en.300.bin')
    word_bad, word_good = base_word, find_antonym_soft(base_word, model)
    neg_end, pos_end = extreme_neighbors(word_bad, word_good, model, scale=10, k=30)
    clean_neg = dedupe_word_list_strong(neg_end, model)
    clean_pos = dedupe_word_list_strong(pos_end, model)
    return clean_neg, clean_pos, word_good

def find_extremes_good(base_word, model):
    # model = fasttext.load_model('cc.en.300.bin')
    word_bad, word_good = base_word, find_antonym_soft(base_word, model)
    neg_end, pos_end = extreme_neighbors(word_bad, word_good, model, scale=10, k=30)
    clean_neg = dedupe_word_list_strong(neg_end, model)
   
    word_bad, word_good = word_good, find_antonym_soft(word_good, model)
    neg_end, pos_end = extreme_neighbors(word_bad, word_good, model, scale=10, k=30)
    clean_pos = dedupe_word_list_strong(neg_end, model)
    return clean_neg, clean_pos, word_bad

def proj_meas(v1, v2, v3):
    dl = np.linalg.norm(v1 - v3)
    dr = np.linalg.norm(v2 - v3)
    dh = np.linalg.norm(v1 - v2)
    
    if dl != 0:
        alpha = np.arccos((dl**2+dh**2-dr**2)/(2*dl*dh))
    else:
        alpha = 0
    proj, d = dl*np.cos(alpha), dl*np.sin(alpha)
    t = proj/dh
    return d, t

def find_adjectives(word, word_list):
    ADJ_SUFFIXES = (
    'ous', 'ful', 'less', 'able', 'ible',
    'ic', 'ical', 'ish', 'ive'
    )
    adjs = []
    for w in word_list:
        if not word in re.sub(r'[^a-zA-Z]', '', w).lower():
            tag = nltk.pos_tag([w])[0][1]
            if tag in ('JJ', 'JJR', 'JJS'):
                adjs.append(w)
            elif w.lower().endswith(ADJ_SUFFIXES):
                adjs.append(w)
            
    return adjs

def create_t_adjusted_list(word, word_list, model, weight=1):
    bad_words, good_words, _ = find_extremes_fast(word, model)
    bad_vecs = np.array([model.get_word_vector(w) for w,_ in bad_words])
    good_vecs = np.array([model.get_word_vector(w) for w,_ in good_words])
    print([w for w,_ in bad_words])
    adjectives = find_adjectives(word, [w for w,_ in bad_words])
    # print(adjectives)
    bad_mean = np.mean(bad_vecs, axis=0)
    good_mean = np.mean(good_vecs, axis=0)

    scale_scores = []
    dist_scores = []

    # Compute t and distance for each word
    for word in word_list:
        deter = model.get_word_vector(word)
        d, t = proj_meas(bad_mean, good_mean, deter)
        scale_scores.append(t)
        dist_scores.append(d)

    scores = np.array(scale_scores)
    dists = np.array(dist_scores)
    words = np.array(word_list)

    normed_dists = (dists - np.min(dists)) / (np.max(dists) - np.min(dists))
    t_avg = np.mean(scores)
    t_adjusted = scores + weight * (t_avg - scores) * normed_dists

    ordered_scores = list(t_adjusted[np.argsort(t_adjusted)])
    ordered_words = list(words[np.argsort(t_adjusted)])

    return ordered_words, ordered_scores, adjectives

def build_table(lists, headers):
    table = []

    # zip(*lists) gives tuples like (list1[i], list2[i], list3[i], ...)
    for row in zip(*lists):
        formatted = []
        for value in row:
            # If it's numeric, format to 3 decimals
            if isinstance(value, (float, int)):
                formatted.append(f"{value:.3f}")
            else:
                formatted.append(value)

        table.append(formatted)

    print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

print('Start Execution')
start_time = time.time()
model = fasttext.load_model('cc.en.300.bin')
ordered_words, ordered_scores, adjectives = create_t_adjusted_list('poor', animals, model)
print('total time: ', time.time()-start_time)
print(adjectives)
print('personal choice adjective is the one with most letters: ', max(adjectives, key=len))
build_table([ordered_words, ordered_scores], ['words', 'scores'])

Start Execution
['poor', 'abysmal', 'inadequate', 'piss-poor', 'sub-standard', 'atrocious', 'sub-par', 'lousy', 'woeful', 'pitiful', 'less-than-adequate', 'unsatisfactory', 'shoddy', 'less-than-stellar', 'worsened', 'bad', 'below-standard', 'suboptimal', 'less-than-good', 'deplorable']
total time:  51.69415044784546
['atrocious', 'woeful', 'pitiful', 'less-than-stellar', 'bad', 'deplorable']
personal choice adjective is the one with most letters:  less-than-stellar
╒════════════════╤══════════╕
│ words          │   scores │
╞════════════════╪══════════╡
│ Crow           │    0.355 │
├────────────────┼──────────┤
│ Duckling       │    0.385 │
├────────────────┼──────────┤
│ Bear           │    0.391 │
├────────────────┼──────────┤
│ Stork          │    0.391 │
├────────────────┼──────────┤
│ Lemming        │    0.399 │
├────────────────┼──────────┤
│ Sloth          │    0.4   │
├────────────────┼──────────┤
│ Wolf           │    0.402 │
├────────────────┼──────────┤
│ Weasel         │  